# Workbench: Data validation

## Setup

In [ ]:
import utils
import polars as pl
import math
from datetime import date
from pathlib import Path

for path in Path("input").glob("*.accdb"):
  df_all = utils.get_df(path)

# for validation involving rows with full IC number
df_full_ic = df_all.filter(pl.col("ICNUMBER").str.contains(r"^\d{12}$"))

# define identifier columns
expr_id_cols = pl.col("DISTRICT", "LOCATION OF SCREENING", "DATESCREEN", "ICNUMBER")

`df`: Full rows  
`df_full_ic`: Rows with full IC number - 12 digits.

## Dates

### [`3 DATEBIRTH` vs `ICNUMBER`](../docs/rules.md#3-datebirth-vs-icnumber)

In [ ]:
# get this year
this_year = date.today().year
this_year_p1 = math.floor(this_year / 100)
this_year_p2 = this_year % 100

In [ ]:
df = (
  df_full_ic.select(
    expr_id_cols,
    "DATEBIRTH",
    pl.col("ICNUMBER").str.slice(0, 2).cast(pl.Int16).alias("R3_year_p2"),
  )
  .with_columns(  # calculate first two digits of birth year from IC
    pl.when(pl.col("R3_year_p2") > this_year_p2)
    .then(this_year_p1 - 1)
    .otherwise(this_year_p1)
    .cast(pl.Utf8)
    .alias("R3_year_p1")
  )
  .with_columns(  # concat the first two digit with IC number to form full date string
    (pl.col("R3_year_p1") + pl.col("ICNUMBER").str.slice(0, 6)).alias("R3_datestr"),
  )
  .with_columns(  # slice the string and cast into date
    pl.concat_str(
      [
        pl.col("R3_datestr").str.slice(0, 4),
        pl.col("R3_datestr").str.slice(4, 2),
        pl.col("R3_datestr").str.slice(6, 2),
      ],
      separator="-",
    )
    .str.to_date()
    .alias("R3_date_from_ic")
  )
  .filter(pl.col("DATEBIRTH") != pl.col("R3_date_from_ic"))
)

df.select(expr_id_cols, R3_fail=1)

### [4 `DATESCREEN` vs `DATE REFERRED` vs `DATE REFERRED QUIT SER`](../docs/rules.md#4-datescreen-vs-date-referred-vs-date-referred-quit-ser)

In [ ]:
(
  df_all.select(expr_id_cols, "DATE REFERRED", "DATE REFERRED QUIT SER").filter(
    (pl.col("DATE REFERRED") < pl.col("DATESCREEN"))
    | (pl.col("DATE REFERRED QUIT SER") < pl.col("DATESCREEN"))
  )
)


### [5 `DATE REFERRED` vs `DATE SEEN BY SPECIALIST`](../docs/rules.md#5-date-referred-vs-date-seen-by-specialist)

In [ ]:
df_all.select(expr_id_cols, "DATE REFERRED", "DATE SEEN BY SPECIALIST").filter(
  (pl.col("DATE SEEN BY SPECIALIST") < pl.col("DATE REFERRED"))
)

### [6 `DATE REFERRED QUIT SER` vs `TARIKH TEMUJANJI QUIT SERVICE`](../docs/rules.md#6-date-referred-quit-ser-vs-tarikh-temujanji-quit-service)

In [ ]:
df_all.select(
  expr_id_cols, "DATE REFERRED QUIT SER", "TARIKH TEMUJANJI QUIT SERVICE"
).filter((pl.col("TARIKH TEMUJANJI QUIT SERVICE") < pl.col("DATE REFERRED QUIT SER")))


## Others

### [1 `ICNUMBER` vs `GENDER`](../docs/rules.md#1-icnumber-vs-gender)

In [ ]:
# validation library prototype

from includes import RuleEnum


def _get_fail_expr(rule_number: RuleEnum, col_list: list) -> pl.Expr:
  return pl.struct(
    pl.lit(rule_number.value).alias("rule"), pl.struct(col_list).alias("data")
  ).alias("fail")


df_validate = pl.DataFrame(
  schema={
    "DISTRICT": pl.Utf8,
    "LOCATION OF SCREENING": pl.Utf8,
    "DATESCREEN": pl.Date,
    "ICNUMBER": pl.Utf8,
    "fail": pl.Struct({"rule": pl.Int16, "data": pl.Struct}),
  }
)

In [ ]:
df = (
  df_full_ic.select(expr_id_cols, "GENDER CODE")
  .with_columns(
    (pl.col("GENDER CODE").cast(pl.Int16) % 2).alias("R1_GENDER_mod"),
    (pl.col("ICNUMBER").str.slice(-1).cast(pl.Int16) % 2).alias("R1_IC_mod"),
  )
  .filter(pl.col("R1_GENDER_mod") != pl.col("R1_IC_mod"))
  .select(
    expr_id_cols,
    _get_fail_expr(RuleEnum.IC_VS_GENDER, ["ICNUMBER", "GENDER CODE"]),
  )
)

df

### [2 `LESION` vs `REFERAL TO SPECIALIST`](../docs/rules.md#2-lesion-vs-referal-to-specialist)

In [ ]:
df_all.select(
  expr_id_cols,
  "LESION",
  "REFERAL TO SPECIALIST",
  pl.when(pl.col("LESION") != pl.col("REFERAL TO SPECIALIST"))
  .then(1)
  .otherwise(0)
  .alias("R2_violation"),
).filter(pl.col("R2_violation") == 1)